# Read & Write Sink Particle Snapshots

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import pycstruct

In [8]:
def read_dump_file(file, bh=False):
    f = open(file, 'rb')
    
    Time = np.fromfile(f, np.float64, 1)
    NSinksAllTasks = np.fromfile(f, np.uint32, 1)
    
    struct = pycstruct.StructDef(alignment = 8)
    struct.add('float64', 'Pos', shape=3)
    struct.add('float64', 'Vel', shape=3)
    struct.add('float64', 'Accel', shape=3)
    struct.add('float64', 'Mass')
    struct.add('float64', 'FormationMass')
    struct.add('float64', 'FormationTime')
    struct.add('uint64', 'ID')
    struct.add('uint32', 'HomeTask')
    struct.add('uint32', 'Index')
    struct.add('uint32', 'FormationOrder')
    if bh == True:
        struct.add('uint32', 'BlackHole')
        struct.add('float64', 'BlackHoleAccretionRadius')
        struct.add('float64', 'BlackHoleMass')
        struct.add('float64', 'BlackHoleDiskGasMass')
        struct.add('float64', 'BlackHoleSinkAccretionRate')
        struct.add('float64', 'BlackHoleAccCellsTotalMass')
    
    inbytes = f.read()
    data = struct.deserialize(inbytes)
    
    f.close()
    
    return Time, NSinksAllTasks, data

def write_dump_file(filename, bh=False):
    struct = pycstruct.StructDef(alignment=8)
    struct.add('float64', 'Pos', shape=3)
    struct.add('float64', 'Vel', shape=3)
    struct.add('float64', 'Accel', shape=3)
    struct.add('float64', 'Mass')
    struct.add('float64', 'FormationMass')
    struct.add('float64', 'FormationTime')
    struct.add('uint64', 'ID')
    struct.add('uint32', 'HomeTask')
    struct.add('uint32', 'Index')
    struct.add('uint32', 'FormationOrder')
    if bh == True:
        struct.add('uint32', 'BlackHole')
        struct.add('float64', 'BlackHoleAccretionRadius')
        struct.add('float64', 'BlackHoleMass')
        struct.add('float64', 'BlackHoleDiskGasMass')
        struct.add('float64', 'BlackHoleSinkAccretionRate')
        struct.add('float64', 'BlackHoleAccCellsTotalMass')

    data = {}
    data['Pos']            = [20, 20, 20]
    data['Vel']            = [0, 0, 0]
    data['Accel']          = [0, 0, 0]
    data['Mass']           = 1e7
    data['FormationMass']  = 1e7
    data['FormationTime']  = 0
    data['ID']             = 110001
    data['HomeTask']       = 0
    data['Index']          = 0
    data['FormationOrder'] = 0
    if bh == True:
        data['BlackHole']                  = 1
        data['BlackHoleAccretionRadius']   = 1.0
        data['BlackHoleMass']              = 1e7
        data['BlackHoleDiskGasMass']       = 0
        data['BlackHoleSinkAccretionRate'] = 0
        data['BlackHoleAccCellsTotalMass'] = 0
    buffer = struct.serialize(data)
    
    f = open(filename, 'wb')
    np.array([0], dtype=np.double).tofile(f)
    np.array([1], dtype=np.intc).tofile(f)
    f.write(buffer)
    f.close()
    
    return 1

def read_hdf5_file(file):
    h = h5py.File(file, 'r')
    
    print('HDF5-file keys:')
    for i in h.keys():
        print(i)
    
    print('\nHeader:')
    for i in h['Header'].attrs:
        print(i, h['Header'].attrs[i])
    
    print('\nPartType5 keys:')
    for i in h['PartType5'].keys():
        print(i, h['PartType5'][i][:])
    
    return 1

## Read Snapshot:

In [11]:
dump = '/hpcstorage/jpeterss/spherecollapse/fiducial/OUTPUT/sink_snap_001'
hdf5 = '/hpcstorage/jpeterss/spherecollapse/fiducial/OUTPUT/snap_001.hdf5'


In [12]:
read_dump_file(dump, bh=False)

(array([1.20214844]),
 array([1], dtype=uint32),
 {'Pos': [21.121470419115532, 4.960931277234721, 15.300402521488106],
  'Vel': [0.00044362805842551584,
   -4.799308752068949e-05,
   -0.001152399840066255],
  'Accel': [0.12145278488061406, 0.006235908266947927, 0.003135396903074486],
  'Mass': 100870.38460174497,
  'FormationMass': 100000.0,
  'FormationTime': 0.0,
  'ID': 110001,
  'HomeTask': 18,
  'Index': 1866,
  'FormationOrder': 0})

In [14]:
read_hdf5_file(hdf5)

HDF5-file keys:
Config
Header
Parameters
PartType0
PartType5

Header:
BoxSize 40.0
Composition_vector_length 0
Flag_Cooling 0
Flag_DoublePrecision 1
Flag_Feedback 0
Flag_Metals 0
Flag_Sfr 0
Flag_StellarAge 0
Git_commit b'd3f7a2da9dad4cca1339f6233f43885006a726e4'
Git_date b'Thu Dec 8 09:48:17 2022 +0100'
HubbleParam 1.0
MassTable [0. 0. 0. 0. 0. 0.]
NumFilesPerSnapshot 1
NumPart_ThisFile [56446     0     0     0     0     1]
NumPart_Total [56446     0     0     0     0     1]
NumPart_Total_HighWord [0 0 0 0 0 0]
Omega0 0.0
OmegaBaryon 0.0
OmegaLambda 0.0
Redshift 0.0
Time 1.2021484375
UnitLength_in_cm 3.08567758e+18
UnitMass_in_g 1.9891e+33
UnitVelocity_in_cm_per_s 100000.0

PartType5 keys:
Coordinates [[19.99993215 20.00002295 19.99996404]]
Masses [100870.38460174]
ParticleIDs [110001]
Velocities [[ 0.0006093  -0.00014908 -0.00126227]]


0

## Write Snapshot:

In [9]:
write_dump_file('sink_snap_000', bh=True)
read_dump_file('sink_snap_000', bh=True)

(array([0.]),
 array([1], dtype=uint32),
 {'Pos': [20.0, 20.0, 20.0],
  'Vel': [0.0, 0.0, 0.0],
  'Accel': [0.0, 0.0, 0.0],
  'Mass': 10000000.0,
  'FormationMass': 10000000.0,
  'FormationTime': 0.0,
  'ID': 110001,
  'HomeTask': 0,
  'Index': 0,
  'FormationOrder': 0,
  'BlackHole': 1,
  'BlackHoleAccretionRadius': 1.0,
  'BlackHoleMass': 10000000.0,
  'BlackHoleDiskGasMass': 0.0,
  'BlackHoleSinkAccretionRate': 0.0,
  'BlackHoleAccCellsTotalMass': 0.0})